In [1761]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [1762]:
from __future__ import division
from pyomo.opt import SolverFactory

In [1763]:
# Reading the data from Excel workbook
InputData = 'C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx'

# Reading data from Cost sheet
cost = pd.read_excel(InputData,sheet_name='Cost')

# Reading data from StaffAvailability sheet
availability = pd.read_excel(InputData,sheet_name='StaffAvailability')

# Reading data from DemandData sheet
demand = pd.read_excel(InputData,sheet_name='DemandData')

# Reading data from ServiceRate sheet
servicerate = pd.read_excel(InputData, sheet_name = 'ServiceRate')

In [1764]:
# Reading Cost sheet

cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost,UnitOutCost_v2
0,A,Jan,60000,5000.0,180,155
1,A,Feb,60000,5000.0,180,155
2,A,Mar,60000,5000.0,180,155
3,A,Apr,60000,5000.0,180,155
4,A,May,60000,5000.0,180,155


In [1765]:
# Reading StaffAvailability sheet

availability.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [1766]:
# Reading DemandData sheet

demand.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [1767]:
# Reading ServiceRate

servicerate.head()

,MgAppServedPerMonth
0,40


## Data pre-processing 

In [1768]:
# Create the required Python data structures for indexes and parameters

# DEMAND-Per month, per state
Demand = demand.set_index(['State', 'Month'])['Demand'].to_dict()
print('Demand : ', Demand)

# Average availability of staff
StaffAvPer = availability.set_index(['State', 'Month'])['StaffAvPer'].to_dict()
print('\n Availability Rate : ', StaffAvPer)

#Availability of employees - best case with high availability - upper bound
StaffAv_UB = availability.set_index(['State', 'Month'])['UB'].to_dict()
print('\n Availability Rate : ', StaffAv_UB)

# Availability of employees - worst case with low availability - lower bound
StaffAv_LB = availability.set_index(['State', 'Month'])['LB'].to_dict()
print('\n Availability Rate : ', StaffAv_LB)

# FTE Salary data
FTE_Salary = cost.set_index(['State', 'Month'])['MonthlySalary'].to_dict()
print('\n Employee Cost : ', FTE_Salary)

# Unit Outsourcing Cost
outsourcecost = cost.set_index(['State', 'Month'])['UnitOutSourceCost'].to_dict()
print('\n Outsource Cost : ', outsourcecost)

# Outosurcing cost per application - Vendor2
outsourcecost_vendor2=cost.set_index(['State', 'Month'])['UnitOutCost_v2'].to_dict()
print('\n Outsource Cost V2 : ', outsourcecost_vendor2)

# Regulatory requirement on maximum outsourcing 
regulation_base ={'A':0.3, 'B':0.4, 'C':1.0}
print('\n Regulatory percent of Outsource allowed : ', regulation_base)

# No. of application handled by FTE on 100% working
FTE_MgAppServed = servicerate.iloc[0,0]
print('\n Service rate) : ', FTE_MgAppServed)

Demand :  {('A', 'Jan'): 5240, ('A', 'Feb'): 4878, ('A', 'Mar'): 5942, ('A', 'Apr'): 2297, ('A', 'May'): 1992, ('A', 'Jun'): 2275, ('A', 'Jul'): 5334, ('A', 'Aug'): 3371, ('A', 'Sep'): 3759, ('A', 'Oct'): 3529, ('A', 'Nov'): 4284, ('A', 'Dec'): 5183, ('B', 'Jan'): 4927, ('B', 'Feb'): 2628, ('B', 'Mar'): 2974, ('B', 'Apr'): 2338, ('B', 'May'): 4020, ('B', 'Jun'): 3147, ('B', 'Jul'): 4271, ('B', 'Aug'): 2620, ('B', 'Sep'): 4517, ('B', 'Oct'): 4155, ('B', 'Nov'): 3137, ('B', 'Dec'): 4227, ('C', 'Jan'): 1162, ('C', 'Feb'): 1967, ('C', 'Mar'): 1898, ('C', 'Apr'): 2261, ('C', 'May'): 2030, ('C', 'Jun'): 1642, ('C', 'Jul'): 2489, ('C', 'Aug'): 2496, ('C', 'Sep'): 922, ('C', 'Oct'): 2421, ('C', 'Nov'): 963, ('C', 'Dec'): 1998}

 Availability Rate :  {('A', 'Jan'): 0.81, ('A', 'Feb'): 0.76, ('A', 'Mar'): 0.75, ('A', 'Apr'): 0.8, ('A', 'May'): 0.78, ('A', 'Jun'): 0.73, ('A', 'Jul'): 0.68, ('A', 'Aug'): 0.76, ('A', 'Sep'): 0.81, ('A', 'Oct'): 0.73, ('A', 'Nov'): 0.68, ('A', 'Dec'): 0.65, ('B', 'J

In [1769]:
# Index

# unique value of State index over each sate
state = cost['State'].unique()
print('state : ', state)

# unique value of time index over each month
month = cost['Month'].unique()
print('month : ', month)

state :  ['A' 'B' 'C']
month :  ['Jan' 'Feb' 'Mar' 'Apr' 'May' 'Jun' 'Jul' 'Aug' 'Sep' 'Oct' 'Nov' 'Dec']


# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [1770]:
# Creating a model instance
model = ConcreteModel()

In [1771]:
# Define Pyomo sets and Parameters

# Index :  month and  state

#Parameters : in code terms it is already defined above
# for in-house processing
#  Employee Cost - Monthly Salary across each state  : employeecost
#  Service Rate - Maximum applications that can be processed per FTE  : FTE_MgAppServed
# Availability Rate - Percentage of avaialability for FTE for each state for each month : StaffAvPer
# for Outsourced processing
# Outsourcing Cost - Outsourced processing cost per application :  outsourcecost
# for contraint definitions
# regulatory limit on percentage of application that can be outsourced : regulation_base
# demand for each state each month that needs to be completely serviced : Demand

In [1772]:
# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced vendor
model.y = Var(state, month, within=Integers, bounds=(0,None))

In [1773]:
# Objective function

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])
               +(model.y[s,m])*outsourcecost[s,m])
                for s in state for m in month)

model.value = Objective(rule=obj_rule, sense= minimize) # Minimize the total cost for the application approval process

In [1774]:
# Constraints
# OUTSOURCE CONSTRAINT - Regulatory requirement of maximum percentage of applications that can be outsourced

def max_outsourced_perstate_permonth(model,s,m):
    return (model.y[s,m]/Demand[s,m]) <= regulation_base[s]
                   
model.max_outsourced = Constraint(state, month, rule=max_outsourced_perstate_permonth)

In [1775]:
# Constraints
# DEMAND CONSTRAINT - entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1776]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.14058923721313477
# ----------------------------------------------------------
#   Solution Information
# -----------

In [1777]:
#dispalying optimal result parameters
model.pprint()

12 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1778]:
# Print the value of the objective function
model.value()

17962336.44876988

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [1779]:
# Creating dataframe for the results

#Creating an empty list
output2 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Processing Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor (Outsourced)
        Outsourced=model.y[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAvPer[s,m],2)
        
        #percentage of application own staff 
        PercentOwnStaff=round(model.x[s,m].value / Demand[s,m]*100,2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m]),2)
                
        output2.append([s,m,Dem,OwnStaff,Outsourced,FTE, PercentOwnStaff, PercentOutsourced,round(ProcessingCost,2)])
        
output2 = pd.DataFrame(output2,columns=['State','Month','Demand','OwnStaff','Outsourced','FTE',\
                                        'PercentOwnStaff','PercentOutsourced','BaseProcesingCost'])
output2

,State,Month,Demand,OwnStaff,Outsourced,FTE,PercentOwnStaff,PercentOutsourced,BaseProcesingCost
0,A,Jan,5240,5240.0,0.0,161.73,100.00,0.00,808641.98
1,A,Feb,4878,4878.0,0.0,160.46,100.00,0.00,802302.63
2,A,Mar,5942,5942.0,0.0,198.07,100.00,0.00,990333.33
3,A,Apr,2297,2297.0,0.0,71.78,100.00,0.00,358906.25
4,A,May,1992,1992.0,0.0,63.85,100.00,0.00,319230.77
5,A,Jun,2275,2275.0,0.0,77.91,100.00,0.00,389554.79
6,A,Jul,5334,3734.0,1600.0,137.28,70.00,30.00,974397.06
7,A,Aug,3371,3371.0,0.0,110.89,100.00,0.00,554440.79
8,A,Sep,3759,3759.0,0.0,116.02,100.00,0.00,580092.59
9,A,Oct,3529,3529.0,0.0,120.86,100.00,0.00,604280.82


In [1780]:
#calcualting Total Processing cost
print(round(sum(output2['BaseProcesingCost'])/1000000,2), "millions")

17.96 millions


In [1781]:
# Its $ 17.96 million

In [1782]:
print(round((sum(output2['BaseProcesingCost']))/sum(output2['Demand']),2), \
      " processing cost per application")

158.55  processing cost per application


In [1783]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data'
output2.to_excel(writer, sheet_name='Question 2')

# save the excel file
writer.save()
print('DataFrame is loaded successfully to Excel Sheet.')

DataFrame is loaded successfully to Excel Sheet.


# Question 3 

#### Expected Scenarios

The company wants to get a solution for the scenario, where no more than 10% of the total applications for every month in each state is handled by the external resources, for quality assurance reasons

- **`Q 3.1-`** What is the additional average cost per application of keeping the 10% limit on outsourcing? 
Note: Compare with the cost in question 2

- **`Q 3.2-`** If the company increases the outsourcing costs by 20% to reflect the cost of quality assurance, what would you recommend the optimal percent of outsourcing should be?



#### Expected output: 
`Subtask 3.1-` Create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the additional average cost per application.

`Subtask 3.2-` Create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the additional average cost per application. Also, print the overall average optimal per cent of outsourcing.

### 3.1 Additional cost per application of keeping the 10% limit on outsourcing

In [1784]:
# Creating a model instance
model = ConcreteModel()

In [1785]:
# Define Pyomo sets and Parameters

#only change will be to quality related contraints rather than regulation

#Quality requirement on maximum outsourcing 
quality_scenario1 ={'A':0.1, 'B':0.1, 'C':0.1}
print('\n Quality reasons percent of Outsource allowed : ', quality_scenario1)


 Quality reasons percent of Outsource allowed :  {'A': 0.1, 'B': 0.1, 'C': 0.1}


In [1786]:
# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced Vendor
model.y = Var(state, month, within=Integers, bounds=(0,None))

In [1787]:
# Constraints

#Quality requirement of maximum percentage of applications that can be outsourced - Scenario 1
def max_outsourced_perstate_permonth(model,s,m):
    return (model.y[s,m]/Demand[s,m]) <= quality_scenario1[s]
                   
model.max_outsourced = Constraint(state, month, rule=max_outsourced_perstate_permonth)

In [1788]:
# Constraints
#entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1789]:
# Objective function

# Defining the objective rule

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])
               +(model.y[s,m])*outsourcecost[s,m])
                for s in state for m in month)

# Minimize the total cost for the application approval process
model.value = Objective(rule=obj_rule, sense= minimize)

In [1790]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 18100979.3212198
  Upper bound: 18100979.3212198
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.14062213897705078
# ----------------------------------------------------------
#   Solution Information
# -----------

In [1791]:
# Model parameters after optimization
model.pprint()

12 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1792]:
# Print the value of the objective function
model.value()

18100979.32121985

**`Checkpoint 2:`** The company has to spend around 18.1 m$ in total for the application approval process if there is 10% limit on the outsourcing.

In [1793]:
# Creating dataframe for the results
#Creating an empty list
output3_1 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Total Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor (Outsourced)
        Outsourced=model.y[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAvPer[s,m],2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m]),2)
                
        output3_1.append([s,m,Dem,OwnStaff,Outsourced,FTE, PercentOutsourced,round(ProcessingCost,2)])
        
output3_1 = pd.DataFrame(output3_1,columns=['State','Month','Demand','OwnStaff','Outsourced','FTE',\
                                            'PercentOutsourced','Cost10%Outsourced'])
output3_1

,State,Month,Demand,OwnStaff,Outsourced,FTE,PercentOutsourced,Cost10%Outsourced
0,A,Jan,5240,5240.0,0.0,161.73,0.00,808641.98
1,A,Feb,4878,4878.0,0.0,160.46,0.00,802302.63
2,A,Mar,5942,5942.0,0.0,198.07,0.00,990333.33
3,A,Apr,2297,2297.0,0.0,71.78,0.00,358906.25
4,A,May,1992,1992.0,0.0,63.85,0.00,319230.77
5,A,Jun,2275,2275.0,0.0,77.91,0.00,389554.79
6,A,Jul,5334,4801.0,533.0,176.51,9.99,978476.76
7,A,Aug,3371,3371.0,0.0,110.89,0.00,554440.79
8,A,Sep,3759,3759.0,0.0,116.02,0.00,580092.59
9,A,Oct,3529,3529.0,0.0,120.86,0.00,604280.82


In [1794]:
# Print the value of the objective function
print(round(sum(output3_1['Cost10%Outsourced']/1000000),3), "millions")

18.101 millions


In [1795]:
# Its $ 18.1 million

In [1796]:
# Additional cost per application

print(round(sum(output3_1['Cost10%Outsourced'])/1000000,2)," millions is total processing cost with 10% Outsourcing limit")
      
print(round((sum(output3_1['Cost10%Outsourced'])-sum(output2['BaseProcesingCost']))/1000000,2), \
      "millions is additional processing cost with 10% Outsourcing limit compared to base case \n")
    
print(round(sum(output2['BaseProcesingCost'])/sum(output2['Demand']),2),\
            " rupees is processing cost per application for Base Case")
      
print(round(sum(output3_1['Cost10%Outsourced'])/sum(output2['Demand']),2), \
      " rupees is processing cost per application with 10% Outsourcing limit \n")

print(round((sum(output3_1['Cost10%Outsourced'])-sum(output2['BaseProcesingCost']))/sum(output2['Demand']),2), \
      " rupees is additional processing cost per application with 10% Outsourcing limit compared to base case")

18.1  millions is total processing cost with 10% Outsourcing limit
0.14 millions is additional processing cost with 10% Outsourcing limit compared to base case 

158.55  rupees is processing cost per application for Base Case
159.77  rupees is processing cost per application with 10% Outsourcing limit 

1.22  rupees is additional processing cost per application with 10% Outsourcing limit compared to base case


In [1797]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data.xlsx'
output3_1.to_excel(writer, sheet_name='Question 3 Scenario 1')

# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


### 3.2  Increase in the outsourcing costs by 20% 

In [1798]:
# Creating a model instance
model = ConcreteModel()

In [1799]:
# Define Pyomo sets and Parameters

# Only thing that changes is the Outsourcing Cost, which will increase by 20 %, for quality assurance.

for i in outsourcecost.keys():
    outsourcecost[i]=1.2*outsourcecost[i]
print(outsourcecost)

{('A', 'Jan'): 216.0, ('A', 'Feb'): 216.0, ('A', 'Mar'): 216.0, ('A', 'Apr'): 216.0, ('A', 'May'): 216.0, ('A', 'Jun'): 216.0, ('A', 'Jul'): 216.0, ('A', 'Aug'): 216.0, ('A', 'Sep'): 216.0, ('A', 'Oct'): 216.0, ('A', 'Nov'): 216.0, ('A', 'Dec'): 216.0, ('B', 'Jan'): 180.0, ('B', 'Feb'): 180.0, ('B', 'Mar'): 180.0, ('B', 'Apr'): 180.0, ('B', 'May'): 180.0, ('B', 'Jun'): 180.0, ('B', 'Jul'): 180.0, ('B', 'Aug'): 180.0, ('B', 'Sep'): 180.0, ('B', 'Oct'): 180.0, ('B', 'Nov'): 180.0, ('B', 'Dec'): 180.0, ('C', 'Jan'): 192.0, ('C', 'Feb'): 192.0, ('C', 'Mar'): 192.0, ('C', 'Apr'): 192.0, ('C', 'May'): 192.0, ('C', 'Jun'): 192.0, ('C', 'Jul'): 192.0, ('C', 'Aug'): 192.0, ('C', 'Sep'): 192.0, ('C', 'Oct'): 192.0, ('C', 'Nov'): 192.0, ('C', 'Dec'): 192.0}


In [1800]:
# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced Vendor
model.y = Var(state, month, within=Integers, bounds=(0,None))

In [1801]:
# Constraints

#Quality requirement of maximum percentage of applications that can be outsourced - Scenario 1
def max_outsourced_perstate_permonth(model,s,m):
    return (model.y[s,m]/Demand[s,m]) <= quality_scenario1[s]
                   
model.max_outsourced = Constraint(state, month, rule=max_outsourced_perstate_permonth)

In [1802]:
#entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1803]:
# Objective function
# Defining the objective rule

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])
               +(model.y[s,m])*outsourcecost[s,m])
                for s in state for m in month)

# Minimize the total cost for the application approval process
model.value = Objective(rule=obj_rule, sense= minimize)

In [1804]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 18144899.6638542
  Upper bound: 18144899.6638542
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.16003656387329102
# ----------------------------------------------------------
#   Solution Information
# -----------

In [1805]:
# Model parameters after optimization
model.pprint()

12 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1806]:
# Print the value of the objective function
model.value()

18144899.66385416

In [1807]:
# Creating dataframe for the results
# Creating dataframe for the results
#Creating an empty list
output3_2 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Total Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor (Outsourced)
        Outsourced=model.y[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAvPer[s,m],2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m]),2)
                
        output3_2.append([s,m,Dem,OwnStaff,Outsourced,FTE, PercentOutsourced,round(ProcessingCost,2)])
        
output3_2 = pd.DataFrame(output3_2,columns=['State','Month','Demand','OwnStaff','Outsourced','FTE',\
                                            'PercentOutsourced','Cost20%Outsourced'])
output3_2

,State,Month,Demand,OwnStaff,Outsourced,FTE,PercentOutsourced,Cost20%Outsourced
0,A,Jan,5240,5240.0,0.0,161.73,0.0,808641.98
1,A,Feb,4878,4878.0,0.0,160.46,0.0,802302.63
2,A,Mar,5942,5942.0,0.0,198.07,0.0,990333.33
3,A,Apr,2297,2297.0,0.0,71.78,0.0,358906.25
4,A,May,1992,1992.0,0.0,63.85,0.0,319230.77
5,A,Jun,2275,2275.0,0.0,77.91,0.0,389554.79
6,A,Jul,5334,5334.0,0.0,196.10,0.0,980514.71
7,A,Aug,3371,3371.0,0.0,110.89,0.0,554440.79
8,A,Sep,3759,3759.0,0.0,116.02,0.0,580092.59
9,A,Oct,3529,3529.0,0.0,120.86,0.0,604280.82


In [1808]:
# Print the value of the objective function
print(round(sum(output3_2['Cost20%Outsourced']/1000000),3), "millions")

18.145 millions


**`Checkpoint 3:`** On increasing the outsourcing cost by 20%, the company has to spend around 18.14 m$, which is slightly more than the previous case .

In [1809]:
# Average optimal percentage of outsourcing 

print(round(sum(output3_2['Cost20%Outsourced'])/1000000,2)," millions is total processing cost with 10% Outsourcing limit and 20% increase in outsource cost")
      
print(round((sum(output3_2['Cost20%Outsourced'])-sum(output2['BaseProcesingCost']))/1000000,2), \
      "millions is total processing cost with 10% Outsourcing limit and 20% increase in outsource cost compared to base case \n")
    
print(round(sum(output2['BaseProcesingCost'])/sum(output2['Demand']),2),\
            " rupees is processing cost per application for Base Case")
      
print(round(sum(output3_2['Cost20%Outsourced'])/sum(output2['Demand']),2), \
      " rupees is processing cost per application with 10% Outsourcing limit and 20% increase in outsource cost \n")

print(round((sum(output3_2['Cost20%Outsourced'])-sum(output2['BaseProcesingCost']))/sum(output2['Demand']),2), \
      " rupees is additional processing cost per application with 10% Outsourcing limit and 20% increase in outsource cost compared to base case")

18.14  millions is total processing cost with 10% Outsourcing limit and 20% increase in outsource cost
0.18 millions is total processing cost with 10% Outsourcing limit and 20% increase in outsource cost compared to base case 

158.55  rupees is processing cost per application for Base Case
160.16  rupees is processing cost per application with 10% Outsourcing limit and 20% increase in outsource cost 

1.61  rupees is additional processing cost per application with 10% Outsourcing limit and 20% increase in outsource cost compared to base case


In [1810]:
# Print the value of the objective function
print(round(sum(output3_2['PercentOutsourced'])))

0


In [1811]:
# Here optimal outsource percentage is 0%.

In [1812]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data.xlsx'
output3_2.to_excel(writer, sheet_name='Question 3 Scenario 2')

# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


# Question 4

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 4.1 Worst case analysis 

- 4.1.1 What is the optimal number of staff members for the worst and best cases? 

- 4.1.2 What is the percentage of outsourcing for the worst and best cases? 

- 4.1.3 What is the average cost per application for the worst and best cases?


#### 4.2 Best case analysis 

- 4.2.1 What is the optimal number of staff members for the worst and best cases? 

- 4.2.2 What is the percentage of outsourcing for the worst and best cases? 

- 4.2.3 What is the average cost per application for the worst and best cases?


#### Expected output:

For each of the subtasks (4.1 and 4.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 4.1 Worst Case Analysis 

#### 4.1.1 Optimal number of staff members

In [1813]:
# Creating a model instance
model = ConcreteModel()

In [1814]:
# Define Pyomo sets and Parameters

# Outsourcing limit for State A is 30% and State B is 40%.

# Outscourcing cost need to remain unchanged which we increased by 20% earlier

for i in outsourcecost.keys():
    outsourcecost[i]=outsourcecost[i]/1.2
print(outsourcecost)

{('A', 'Jan'): 180.0, ('A', 'Feb'): 180.0, ('A', 'Mar'): 180.0, ('A', 'Apr'): 180.0, ('A', 'May'): 180.0, ('A', 'Jun'): 180.0, ('A', 'Jul'): 180.0, ('A', 'Aug'): 180.0, ('A', 'Sep'): 180.0, ('A', 'Oct'): 180.0, ('A', 'Nov'): 180.0, ('A', 'Dec'): 180.0, ('B', 'Jan'): 150.0, ('B', 'Feb'): 150.0, ('B', 'Mar'): 150.0, ('B', 'Apr'): 150.0, ('B', 'May'): 150.0, ('B', 'Jun'): 150.0, ('B', 'Jul'): 150.0, ('B', 'Aug'): 150.0, ('B', 'Sep'): 150.0, ('B', 'Oct'): 150.0, ('B', 'Nov'): 150.0, ('B', 'Dec'): 150.0, ('C', 'Jan'): 160.0, ('C', 'Feb'): 160.0, ('C', 'Mar'): 160.0, ('C', 'Apr'): 160.0, ('C', 'May'): 160.0, ('C', 'Jun'): 160.0, ('C', 'Jul'): 160.0, ('C', 'Aug'): 160.0, ('C', 'Sep'): 160.0, ('C', 'Oct'): 160.0, ('C', 'Nov'): 160.0, ('C', 'Dec'): 160.0}


In [1815]:
# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced vendor
model.y = Var(state, month, within=Integers, bounds=(0,None))

In [1816]:
# Objective function

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAv_LB[s,m])
               +(model.y[s,m])*outsourcecost[s,m])
                for s in state for m in month)

model.value = Objective(rule=obj_rule, sense= minimize) # Minimize the total cost for the application approval process

In [1817]:
# Constraints

#Regulatory requirement of maximum percentage of applications that can be outsourced

def max_outsourced_perstate_permonth(model,s,m):
    return (model.y[s,m]/Demand[s,m]) <= regulation_base[s]
                   
model.max_outsourced = Constraint(state, month, rule=max_outsourced_perstate_permonth)

In [1818]:
# Constraints

#entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1819]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.12562918663024902
# ----------------------------------------------------------
#   Solution Information
# -----------

In [1820]:
# Model parameters after optimization
model.pprint()

12 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1821]:
# Print the value of the objective function
model.value()

19599482.516788762

**`Checkpoint 4:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [1822]:
# Creating dataframe for the results

#Creating an empty list
output4_1 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Total Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor (Outsourced)
        Outsourced=model.y[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAv_LB[s,m],2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAv_LB[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m]),2)
                
        output4_1.append([s,m,Dem,OwnStaff,Outsourced,FTE, PercentOutsourced,round(ProcessingCost,2)])
        
output4_1 = pd.DataFrame(output4_1,columns=['State','Month','Demand','OwnStaff','Outsourced','FTE',\
                                            'PercentOutsourced','Application_Cost'])
output4_1

,State,Month,Demand,OwnStaff,Outsourced,FTE,PercentOutsourced,Application_Cost
0,A,Jan,5240,5240.0,0.0,187.14,0.00,935714.29
1,A,Feb,4878,3415.0,1463.0,131.35,29.99,920070.77
2,A,Mar,5942,5942.0,0.0,212.21,0.00,1061071.43
3,A,Apr,2297,2297.0,0.0,76.57,0.00,382833.33
4,A,May,1992,1992.0,0.0,71.14,0.00,355714.29
5,A,Jun,2275,1593.0,682.0,61.27,29.98,429106.15
6,A,Jul,5334,3734.0,1600.0,155.58,30.00,1065916.67
7,A,Aug,3371,2360.0,1011.0,90.77,29.99,635826.15
8,A,Sep,3759,3759.0,0.0,134.25,0.00,671250.00
9,A,Oct,3529,2471.0,1058.0,95.04,29.98,665632.31


In [1823]:
# Print the value of the objective function
print(round(sum(output4_1['Application_Cost']/1000000),3), "millions")

19.599 millions


#### 4.1.2 Percentage of outsourced applications 

In [1824]:
# write your code here

# Percentage of outsource application
print(round(sum(output4_1['PercentOutsourced']/36),3))

38.604


#### 4.1.3 Average cost per application

In [1825]:
# write your code here

print(round(sum(output4_1['Application_Cost'])/sum(output4_1['Demand']),2), \
      " rupees is average cost per application in worst case. \n")

173.0  rupees is average cost per application in worst case. 



#### Optimal number of staff members for the worst cases

In [1826]:
# Finding optimal number of inhouse staff members for all the state combined in the worst-case

print(sum(output4_1['OwnStaff']))

73486.0


In [1827]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data.xlsx'
output4_1.to_excel(writer, sheet_name='Question 4 Scenario 1')

# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


### 4.2  Best Case Analysis 

#### 4.2.1 Optimal number of staff members


In [1828]:
# Creating a model instance
model = ConcreteModel()

In [1829]:
# Define Pyomo sets and Parameters

# ** Employee Cost - Monthly Salary across each state  : employeecost
# ** Service Rate - Maximum applications that can be processed per FTE  : FTE_MgAppServed
# ** Availability Rate - Percentage of avaialability for FTE for each state for each month : StaffAv_UB
# for Outsourced processing
# ** Outsourcing Cost - Outsourced processing cost per application :  outsourcecost
# for contraint definitions
# ** regulatory limit on percentage of application that can be outsourced : regulation_base
# ** demand for each state each month that needs to be completely serviced : Demand

In [1830]:
# Decision variables

# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced vendor
model.y = Var(state, month, within=Integers, bounds=(0,None))

In [1831]:
# Objective function

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAv_UB[s,m])
               +(model.y[s,m])*outsourcecost[s,m])
                for s in state for m in month)

model.value = Objective(rule=obj_rule, sense= minimize) # Minimize the total cost for the application approval process

In [1832]:
# Constraints

#Regulatory requirement of maximum percentage of applications that can be outsourced

def max_outsourced_perstate_permonth(model,s,m):
    return (model.y[s,m]/Demand[s,m]) <= regulation_base[s]
                   
model.max_outsourced = Constraint(state, month, rule=max_outsourced_perstate_permonth)

In [1833]:
# Constraints

#entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1834]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.14061188697814941
# ----------------------------------------------------------
#   Solution Information
# -----------

In [1835]:
# Model parameters after optimization
model.pprint()

12 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1836]:
# Print the value of the objective function
model.value()

16527535.637935732

**`Checkpoint 5:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [1837]:
# Creating dataframe for the results

#Creating an empty list
output4_2 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Total Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor (Outsourced)
        Outsourced=model.y[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAv_UB[s,m],2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAv_UB[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m]),2)
                
        output4_2.append([s,m,Dem,OwnStaff,Outsourced,FTE, PercentOutsourced,round(ProcessingCost,2)])
        
output4_2 = pd.DataFrame(output4_2,columns=['State','Month','Demand','OwnStaff','Outsourced','FTE',\
                                            'PercentOutsourced','Application_Cost'])
output4_2

,State,Month,Demand,OwnStaff,Outsourced,FTE,PercentOutsourced,Application_Cost
0,A,Jan,5240,5240.0,0.0,145.56,0.00,727777.78
1,A,Feb,4878,4878.0,0.0,143.47,0.00,717352.94
2,A,Mar,5942,5942.0,0.0,185.69,0.00,928437.50
3,A,Apr,2297,2297.0,0.0,67.56,0.00,337794.12
4,A,May,1992,1992.0,0.0,58.59,0.00,292941.18
5,A,Jun,2275,2275.0,0.0,71.09,0.00,355468.75
6,A,Jul,5334,5334.0,0.0,177.80,0.00,889000.00
7,A,Aug,3371,3371.0,0.0,99.15,0.00,495735.29
8,A,Sep,3759,3759.0,0.0,104.42,0.00,522083.33
9,A,Oct,3529,3529.0,0.0,110.28,0.00,551406.25


In [1838]:
# Print the value of the objective function
print(round(sum(output4_2['Application_Cost']/1000000),3), "millions")

16.528 millions


#### 4.2.2 Percentage of outsourced applications

In [1839]:
# write your code here

# Percentage of outsource application
print(round(sum(output4_2['PercentOutsourced']/36),3))

3.332


#### 4.2.3 Average cost per application

In [1840]:
# write your code here

print(round(sum(output4_2['Application_Cost'])/sum(output4_2['Demand']),2), \
      " rupees is average cost per application in worst case. \n")

145.88  rupees is average cost per application in worst case. 



#### Optimal number of staff members for the best cases

In [1841]:
# Finding optimal number of inhouse staff members for all the state combined in the worst case

print(sum(output4_2['OwnStaff']))

108642.0


In [1842]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data.xlsx'
output4_2.to_excel(writer, sheet_name='Question 4 Scenario 2')

# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.


# Question 5

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2, 3 and 4 (i.e., the best case, expected, expected with 10% outsourcing limit and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

# Question 6 

#### Additional proposed outsourced vendor 

The company sees the additional proposed outsourced vendor as ‘VENDOR 2’ with a cost per application as given below.

**`Vendor 1`**	
**A:** 180 Dollars |
**B:** 150 Dollars |
**C:** 160 Dollars 

**`Vendor 2`** 	
**A:** 155 Dollars |
**B:** 150 Dollars |
**C:** 158 Dollars

                                                                 

The states have signed a memorandum with both vendors to maintain the ratio (60%: 40%) of Vendor 1:Vendor 2 for the number of applications that can be outsourced for every month in each state.


**`Q 6.1-`** Calculate the optimal staffing at a 10% limit on outsourcing while maintaining the vendor ratio (60%:40%) for the number of applications that can be outsourced for every month in each state.


**`Q 6.2-`** Optimal Allocation of the application after relaxing the ratio criterion mentioned above (60%: 40%) for the number of applications that can be outsourced for every month in each state.


#### Expected output: 
For each subtask (a) and (b), create a data frame containing the number of outsourced applications for each vendor and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier.


###  6.1 Optimal staffing at 10% limit on outsourcing while maintaining the vendor ratio as 60%:40% of the total application

In [1843]:
# Creating a model instance
model = ConcreteModel()

In [1844]:
# Define Pyomo sets and Parameters

#outsourcecost = for vendor_1
#outsourcecost_vendor2 = for vendor_2

# Optimal staffing at 10% limit on outsourcing
outsource_limit ={'A':0.1, 'B':0.1, 'C':0.1}
print('\n Quality reasons percent of Outsource allowed : ', outsource_limit)


 Quality reasons percent of Outsource allowed :  {'A': 0.1, 'B': 0.1, 'C': 0.1}


In [1845]:
# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced Vendor_1
model.y = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced Vendor_2
model.z = Var(state, month, within=Integers, bounds=(0,None))

###### The states have signed a memorandum with both vendors to maintain the Vendor 1:Vendor 2 ratio at 60%:40% for the number of applications that can be outsourced for every month in each state.

In [1846]:
# Constraints

#Quality requirement of maximum percentage of applications that can be outsourced - Scenario 1
def max_outsourced_perstate_permonth(model,s,m):
    return (model.y[s,m]+model.z[s,m]/Demand[s,m]) <= outsource_limit[s]
                   
model.max_outsourced = Constraint(state, month, rule=max_outsourced_perstate_permonth)

In [1847]:
# Constraints
#entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]+model.z[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1848]:
# Vendor 1:Vendor 2 ratio at 60%:40% for the number of applications that can be outsourced for every month in each state.

def ratio(model,s,m):
    return (0.60*model.y[s,m]==0.40*model.z[s,m])
                   
model.ratio_distribution = Constraint(state, month, rule=ratio)

In [1849]:
# Objective function

# Defining the objective rule

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])
               +(model.y[s,m])*outsourcecost[s,m]+(model.z[s,m]*outsourcecost_vendor2[s,m]))
                for s in state for m in month)

# Minimize the total cost for the application approval process
model.value = Objective(rule=obj_rule, sense= minimize)

In [1850]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 18144899.6638542
  Upper bound: 18144899.6638542
  Number of objectives: 1
  Number of constraints: 109
  Number of variables: 109
  Number of nonzeros: 253
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.11744570732116699
# ----------------------------------------------------------
#   Solution Information
# ---------

In [1851]:
#Display model parameters after optimization
model.pprint()

18 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1852]:
# Print the value of the objective function
val = model.value()
print(val/1000000)

18.144899663854158


**`Checkpoint 6:`** The company has to spend around 18.1 m$ in total for the application approval process if there is ratio criterion.

In [1853]:
# Creating dataframe for the results


#Creating an empty list
output6_1 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Total Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor_1 (Outsourced)
        Outsourced_Vendor_1=model.y[s,m].value
        
        #count of applications managed by outsourced vendor_2 (Outsourced)
        Outsourced_Vendor_2=model.z[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAvPer[s,m],2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value+model.z[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAv_UB[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m])+(model.z[s,m].value*outsourcecost_vendor2[s,m]),2)
                
        output6_1.append([s,m,Dem,OwnStaff,Outsourced_Vendor_1, Outsourced_Vendor_2, FTE, PercentOutsourced,ProcessingCost])
        
        
output6_1 = pd.DataFrame(output6_1,columns=['State','Month','Demand','OwnStaff','Outsourced_Vendor1','Outsourced_Vendor2','FTE',\
                                            'PercentOutsourced','Application_Cost'])
output6_1

,State,Month,Demand,OwnStaff,Outsourced_Vendor1,Outsourced_Vendor2,FTE,PercentOutsourced,Application_Cost
0,A,Jan,5240,5240.0,0.0,0.0,161.73,0.0,727777.78
1,A,Feb,4878,4878.0,0.0,0.0,160.46,0.0,717352.94
2,A,Mar,5942,5942.0,0.0,0.0,198.07,0.0,928437.50
3,A,Apr,2297,2297.0,0.0,0.0,71.78,0.0,337794.12
4,A,May,1992,1992.0,0.0,0.0,63.85,0.0,292941.18
5,A,Jun,2275,2275.0,0.0,0.0,77.91,0.0,355468.75
6,A,Jul,5334,5334.0,0.0,0.0,196.10,0.0,889000.00
7,A,Aug,3371,3371.0,0.0,0.0,110.89,0.0,495735.29
8,A,Sep,3759,3759.0,0.0,0.0,116.02,0.0,522083.33
9,A,Oct,3529,3529.0,0.0,0.0,120.86,0.0,551406.25


In [1854]:
# Optimal staffing

sum(output6_1['OwnStaff'])

113294.0

In [1855]:
# Optimal staffing at 10% is 113294.

In [1856]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data'
output6_1.to_excel(writer, sheet_name='Question 6, Scenario 1')

# save the excel file
writer.save()
print('DataFrame is loaded successfully to Excel Sheet.')

DataFrame is loaded successfully to Excel Sheet.


### 6.2	Optimal allocation of application to vendors after relaxing the ratio criterion 60%:40% (Vendor 1:Vendor 2) 

In [1857]:
# Creating a model instance
model = ConcreteModel()

In [1858]:
# Define Pyomo sets and Parameters

#outsourcecost = for vendor_1
#outsourcecost_vendor2 = for vendor_2

In [1859]:
# Decision variables

#Number of applications managed by Staff or own employee
model.x = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced Vendor_1
model.y = Var(state, month, within=Integers, bounds=(0,None))

#Number of applications managed by Outsourced Vendor_2
model.z = Var(state, month, within=Integers, bounds=(0,None))

In [1860]:
# In this scenario there wont be any limit on the outsourcing of the application to the vendor.

# We will also relax the constraint of the 60:40.

In [1861]:
# Constraints
#entire demand should be serviced

def demand_serviced(model,s,m):
    return (model.x[s,m]+ model.y[s,m]+model.z[s,m]) == Demand[s,m]
                   
model.demandserviced = Constraint(state, month, rule=demand_serviced)

In [1862]:
# Objective function

# Defining the objective rule

#Total processing cost = Salary cost for inhouse processed applications + processing cost for outsourced applications
#outsourcing cost is equal to outsourced applications count (Y) mutplied by outsourcing cost per application
#salary cost is decision variable (X)  * employee salary adjusted for servicerate and availability rate

def obj_rule(model):
    
    return sum(((model.x[s,m]*FTE_Salary[s,m]/FTE_MgAppServed/StaffAvPer[s,m])
               +(model.y[s,m])*outsourcecost[s,m]+(model.z[s,m]*outsourcecost_vendor2[s,m]))
                for s in state for m in month)

# Minimize the total cost for the application approval process
model.value = Objective(rule=obj_rule, sense= minimize)

In [1863]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17069134.0056925
  Upper bound: 17069134.0056925
  Number of objectives: 1
  Number of constraints: 37
  Number of variables: 109
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.07812047004699707
# ----------------------------------------------------------
#   Solution Information
# ----------

In [1864]:
#Display model parameters after optimization
model.pprint()

12 Set Declarations
    demandserviced_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                                        : Size : Members
        None :     2 : demandserviced_index_0*demandserviced_index_1 :   36 : {('A', 'Apr'), ('A', 'Aug'), ('A', 'Dec'), ('A', 'Feb'), ('A', 'Jan'), ('A', 'Jul'), ('A', 'Jun'), ('A', 'Mar'), ('A', 'May'), ('A', 'Nov'), ('A', 'Oct'), ('A', 'Sep'), ('B', 'Apr'), ('B', 'Aug'), ('B', 'Dec'), ('B', 'Feb'), ('B', 'Jan'), ('B', 'Jul'), ('B', 'Jun'), ('B', 'Mar'), ('B', 'May'), ('B', 'Nov'), ('B', 'Oct'), ('B', 'Sep'), ('C', 'Apr'), ('C', 'Aug'), ('C', 'Dec'), ('C', 'Feb'), ('C', 'Jan'), ('C', 'Jul'), ('C', 'Jun'), ('C', 'Mar'), ('C', 'May'), ('C', 'Nov'), ('C', 'Oct'), ('C', 'Sep')}
    demandserviced_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    demandserviced_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen :

In [1865]:
# Print the value of the objective function
model.value()

17069134.00569254

**`Checkpoint 7:`** The company has to spend around 17.1 m$ in total for the application approval process on realxing the ratio criterion which is lesser than the previous case. 

In [1866]:
# Creating dataframe for the results

#Creating an empty list
output6_2 = []

# For every iteration, the code below will create a list containing 6 values - 
# 1. count of applications managed by own staff (OwnStaff)
# 2. count of application managed by outsourced vendor (Outsourced)
# 3. demand (Demand) for each state for each month
# 4. number of FTE (own staff required)
# 5. percentage of application outosurced (PercentOutsourced)
# 6. cost of application processing (Total Cost)

for s in state:
    for m in month:
        
        #Demand for each state for each month
        Dem=Demand[s,m]
       
        #count of applications managed by own staff (OwnStaff)
        OwnStaff=model.x[s,m].value
        
        #count of applications managed by outsourced vendor_1 (Outsourced)
        Outsourced_Vendor_1=model.y[s,m].value
        
        #count of applications managed by outsourced vendor_2 (Outsourced)
        Outsourced_Vendor_2=model.z[s,m].value
        
        #Number of FTE
        FTE=round(model.x[s,m].value /FTE_MgAppServed/StaffAvPer[s,m],2)
        
        #percentage of application outosurced (PercentOutsourced)
        PercentOutsourced=round(model.y[s,m].value+model.z[s,m].value / Demand[s,m]*100,2)
        
        
        #cost of application processing (Total Cost)
        ProcessingCost=round((model.x[s,m].value*FTE_Salary[s,m]/FTE_MgAppServed/StaffAv_UB[s,m])\
                        +(model.y[s,m].value*outsourcecost[s,m])+(model.z[s,m].value*outsourcecost_vendor2[s,m]),2)
                
        output6_2.append([s,m,Dem,OwnStaff,Outsourced_Vendor_1, Outsourced_Vendor_2, FTE, PercentOutsourced,ProcessingCost])
        
        
output6_2 = pd.DataFrame(output6_2,columns=['State','Month','Demand','OwnStaff','Outsourced_Vendor1','Outsourced_Vendor2','FTE',\
                                            'PercentOutsourced','Application_Cost'])
output6_2

,State,Month,Demand,OwnStaff,Outsourced_Vendor1,Outsourced_Vendor2,FTE,PercentOutsourced,Application_Cost
0,A,Jan,5240,5240.0,0.0,0.0,161.73,0.0,727777.78
1,A,Feb,4878,0.0,0.0,4878.0,0.00,100.0,756090.00
2,A,Mar,5942,0.0,0.0,5942.0,0.00,100.0,921010.00
3,A,Apr,2297,0.0,0.0,2297.0,0.00,100.0,356035.00
4,A,May,1992,0.0,0.0,1992.0,0.00,100.0,308760.00
5,A,Jun,2275,0.0,0.0,2275.0,0.00,100.0,352625.00
6,A,Jul,5334,0.0,0.0,5334.0,0.00,100.0,826770.00
7,A,Aug,3371,0.0,0.0,3371.0,0.00,100.0,522505.00
8,A,Sep,3759,3759.0,0.0,0.0,116.02,0.0,522083.33
9,A,Oct,3529,0.0,0.0,3529.0,0.00,100.0,546995.00


##### Allocation of application inhouse

In [1867]:
# allocation of application in house

print(sum(output6_2['OwnStaff']))

41600.0


##### Allocation of application to Outsourced_Vendor1

In [1868]:
print(sum(output6_2['Outsourced_Vendor1']))

0.0


##### Allocation of application to Outsourced_Vendor2

In [1869]:
print(sum(output6_2['Outsourced_Vendor2']))

71694.0


In [1870]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("C:\\Users\\dell\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data'
output6_2.to_excel(writer, sheet_name='Question 6, Scenario 2')

# save the excel file
writer.save()
print('DataFrame is loaded successfully to Excel Sheet.')

DataFrame is loaded successfully to Excel Sheet.
